#Update for Complete Data

---

###Schema.txt

* **user_id**: long, user identify,
* **location**: string, the physical location where users employ mobile services,
* **start_time**: long, the UNIX start timestamp of certain user session,
* **session_duration**: long, the temporal duration of certain user session,
* **service_provider**: string, the service provider name,
* **service_category**: string, the service cateogiry classified via domain name,
* **service_domain**: string, the first level domain,
* **bytes**: long, the data sent in bytes of certain user session,
* **requests**: long, the number of individual HTTP requests of certain user session,

###toDb.py

In [17]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine 
import datetime as dt
from IPython.display import display

import plotly.plotly as py 

#plotly.tools.set_credentials_file(username='zedyang', 
#                                  api_key='dyytlcpib3', 
#                                  stream_ids=['436iwi32ro', 
#                                              'sd3b8v5ucf'])

from plotly.graph_objs import Bar, Scatter, Marker, Layout 

In [2]:
display(pd.read_csv('net_traffic.csv', nrows=3).head())
display(pd.read_csv('net_traffic.csv', nrows=3).tail())

,11000752,东中院二区,1411439886390,14799,百度1,搜索导航,baidu.com,2182,2
0,11000752,东中院二区,1411442145178,203010,网易;百度1;谷歌广告1;126网易免费邮箱;谷歌分析;谷歌链接;谷歌搜索2;个性化谷歌广告...,门户;搜索导航;广告;电子邮箱;数码科技;搜索导航;搜索导航;广告;电子邮箱;门户,163.com;baidu.com;doubleclick.net;126.com;goog...,261571;172538;34719;33058;18386;12952;1668;142...,39;33;5;5;2;2;1;1;1;1
1,11000752,东中院三区,1411442842421,0,腾讯微信,即时通讯,qq.com,395,1
2,11000752,学生事务中心-铁生馆,1411443156277,6523,百度1,搜索导航,baidu.com,15887,2


,11000752,东中院二区,1411439886390,14799,百度1,搜索导航,baidu.com,2182,2
0,11000752,东中院二区,1411442145178,203010,网易;百度1;谷歌广告1;126网易免费邮箱;谷歌分析;谷歌链接;谷歌搜索2;个性化谷歌广告...,门户;搜索导航;广告;电子邮箱;数码科技;搜索导航;搜索导航;广告;电子邮箱;门户,163.com;baidu.com;doubleclick.net;126.com;goog...,261571;172538;34719;33058;18386;12952;1668;142...,39;33;5;5;2;2;1;1;1;1
1,11000752,东中院三区,1411442842421,0,腾讯微信,即时通讯,qq.com,395,1
2,11000752,学生事务中心-铁生馆,1411443156277,6523,百度1,搜索导航,baidu.com,15887,2


In [5]:
!wc -l < net_traffic.csv # Number of lines in dataset

 12736408


In [3]:
disk_engine = create_engine('sqlite:///net_traffic.db')

In [49]:
start = dt.datetime.now()
chunkfirst = 1
chunksize = 20000
j = 0
index_start = 1

for df in pd.read_csv('net_traffic.csv', chunksize=chunksize, 
                      iterator=True, encoding='utf-8'):
    
    df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) # Remove spaces from columns
    df.index += index_start
    df.columns=['ID','Location','Start','Duration','Provider',
                'Type','Domain','Size','Req']
        
    j+=1
    print '{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize)

    df.to_sql('data', disk_engine, if_exists='append')
    index_start = df.index[-1] + 1

0 seconds: completed 20000 rows
1 seconds: completed 40000 rows
2 seconds: completed 60000 rows
3 seconds: completed 80000 rows
4 seconds: completed 100000 rows
5 seconds: completed 120000 rows
6 seconds: completed 140000 rows
7 seconds: completed 160000 rows
8 seconds: completed 180000 rows
9 seconds: completed 200000 rows
10 seconds: completed 220000 rows
11 seconds: completed 240000 rows
12 seconds: completed 260000 rows
13 seconds: completed 280000 rows
14 seconds: completed 300000 rows
15 seconds: completed 320000 rows
16 seconds: completed 340000 rows
18 seconds: completed 360000 rows
19 seconds: completed 380000 rows
20 seconds: completed 400000 rows
21 seconds: completed 420000 rows
22 seconds: completed 440000 rows
23 seconds: completed 460000 rows
24 seconds: completed 480000 rows
26 seconds: completed 500000 rows
27 seconds: completed 520000 rows
28 seconds: completed 540000 rows
29 seconds: completed 560000 rows
30 seconds: completed 580000 rows
31 seconds: completed 600000

###summaryData.py

In [4]:
df = pd.read_sql_query('SELECT * FROM data LIMIT 3', disk_engine)
df.head()

,index,ID,Location,Start,Duration,Provider,Type,Domain,Size,Req
0,1,11000752,东中院二区,1411442145178,203010,网易;百度1;谷歌广告1;126网易免费邮箱;谷歌分析;谷歌链接;谷歌搜索2;个性化谷歌广告...,门户;搜索导航;广告;电子邮箱;数码科技;搜索导航;搜索导航;广告;电子邮箱;门户,163.com;baidu.com;doubleclick.net;126.com;goog...,261571;172538;34719;33058;18386;12952;1668;142...,39;33;5;5;2;2;1;1;1;1
1,2,11000752,东中院三区,1411442842421,0,腾讯微信,即时通讯,qq.com,395,1
2,3,11000752,学生事务中心-铁生馆,1411443156277,6523,百度1,搜索导航,baidu.com,15887,2


In [51]:
df = pd.read_sql_query('SELECT ID, Location, Start, Duration '
                       'FROM data '
                       'LIMIT 10', disk_engine)
df

,ID,Location,Start,Duration
0,11000752,东中院二区,1411442145178,203010
1,11000752,东中院三区,1411442842421,0
2,11000752,学生事务中心-铁生馆,1411443156277,6523
3,11000752,学生宿舍W,1411573440149,55
4,11000752,东中院二区,1411956941699,335969
5,11000752,东中院二区,1411957618285,301945
6,11000752,媒体与设计实验室B楼,1413176420667,3230202
7,11000752,媒体与设计实验室B楼,1413180046636,1296008
8,11000752,东中院二区,1413249030600,128279
9,11000752,东中院二区,1413249824366,938


In [5]:
df = pd.read_sql_query('SELECT * '
                       'FROM data '
                       'WHERE Location IN ("东上院")'
                       'LIMIT 10', disk_engine)
df.head()

,index,ID,Location,Start,Duration,Provider,Type,Domain,Size,Req
0,43,11000752,东上院,1415180195500,29395,百度1,搜索导航,baidu.com,2716,4
1,399,11001313,东上院,1420518847989,383,苹果官网1,网上购物,apple.com,7267,1
2,490,11003224,东上院,1415790139482,19158,淘宝网,网上购物,taobao.com,2850,2
3,491,11003224,东上院,1415792211375,4933,淘宝网,网上购物,taobao.com,2838,2
4,492,11003224,东上院,1415793963753,617121,腾讯微信;淘宝网,即时通讯;网上购物,qq.com;taobao.com,13898;2842,7;2


In [53]:
df = pd.read_sql_query('SELECT * '
                       'FROM data '
                       'WHERE ID IN ("11000752")'
                       'LIMIT 10', disk_engine)
df.head()

,index,ID,Location,Start,Duration,Provider,Type,Domain,Size,Req
0,1,11000752,东中院二区,1411442145178,203010,网易;百度1;谷歌广告1;126网易免费邮箱;谷歌分析;谷歌链接;谷歌搜索2;个性化谷歌广告...,门户;搜索导航;广告;电子邮箱;数码科技;搜索导航;搜索导航;广告;电子邮箱;门户,163.com;baidu.com;doubleclick.net;126.com;goog...,261571;172538;34719;33058;18386;12952;1668;142...,39;33;5;5;2;2;1;1;1;1
1,2,11000752,东中院三区,1411442842421,0,腾讯微信,即时通讯,qq.com,395,1
2,3,11000752,学生事务中心-铁生馆,1411443156277,6523,百度1,搜索导航,baidu.com,15887,2
3,4,11000752,学生宿舍W,1411573440149,55,腾讯微信,即时通讯,qq.com,1150,1
4,5,11000752,东中院二区,1411956941699,335969,百度1;携程;携程;谷歌搜索2;谷歌分析;腾讯微信;谷歌广告1,搜索导航;旅游;旅游;搜索导航;数码科技;即时通讯;广告,baidu.com;c-ctrip.com;ctrip.com;google.com.hk;...,248455;132759;107146;13408;12705;3533;2260,88;29;13;13;3;3;2


In [8]:
df = pd.read_sql_query('SELECT DISTINCT Location FROM data', disk_engine)
df.head()
df.tail()

,Location
93,电信群楼5号楼
94,电话总机房
95,法华1号楼
96,法华4号楼
97,七宝网络中心


In [7]:
df

,Location
0,东中院二区
1,东中院三区
2,学生事务中心-铁生馆
3,学生宿舍W
4,媒体与设计实验室B楼
5,电信群楼1号楼
6,棒球场
7,逸夫科技楼
8,第一食堂
9,东上院


In [9]:
df = pd.read_sql_query('SELECT Location, COUNT(*) as `count_login`'
                       'FROM data '
                       'GROUP BY Location ', disk_engine)

df.head()

,Location,count_login
0,null,358556
1,七宝网络中心,245
2,东上院,766678
3,东下院,870668
4,东中院一区,269822


In [19]:
df = pd.read_sql_query('SELECT Location, COUNT(*) as `num_connection`'
                       'FROM data '
                       'GROUP BY Location '
                       'ORDER BY -num_connection', disk_engine)

countByLocation_All = df
df = df[:20]
countByLocation_20 = df
df

,Location,num_connection
0,东下院,870668
1,东上院,766678
2,第二食堂,755220
3,徐汇新上院,753400
4,徐汇工程馆,535833
5,东中院二区,467882
6,陈瑞球北楼,417893
7,东中院四区,407189
8,东中院三区,406937
9,null,358556


####Count By Location
Top 45 Places to connect Wifi.

In [20]:
countByLocation_45 = countByLocation_All[:45]

py.iplot({
    'data': [Bar(x=countByLocation_45['Location'], 
                 y=countByLocation_45.num_connection)],
    'layout': { 
        'margin': {'b': 160}, 
        'xaxis': {'tickangle': 35}} })

####Count By Person
Top 45 students connecting to Wifi most heavily

In [42]:
df = pd.read_sql_query('SELECT ID, COUNT(*) as `num_connection`'
                       'FROM data '
                       'GROUP BY ID '
                       'ORDER BY -num_connection', disk_engine)
df[['ID']]=df[['ID']].astype(object)
countByPerson_All = df
df = df[:45]
countByPerson_45 = df
df

,ID,num_connection
0,32122246,54081
1,19139158,40990
2,93172608,28748
3,61183609,18001
4,13140686,17694
5,35182544,12944
6,25160067,11216
7,23109382,9762
8,61129214,9755
9,63165440,8802


In [44]:
countByPerson_45[['ID']]=countByPerson_45[['ID']].astype(str)
countByPerson_45['ID']='ind_'+countByPerson_45['ID'] #converting bug...
py.iplot({
    'data': [Bar(x=countByPerson_45['ID'], 
    			 y=countByPerson_45.num_connection)],
    'layout': { 
        'margin': {'b': 160}, 
        'xaxis': {'tickangle': 35}} })

（前面那几个几十万的好叼。。